In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os,sys
import errno
import string
from nltk.corpus import reuters 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score,accuracy_score,roc_auc_score,mean_absolute_error
from sklearn.preprocessing import label_binarize
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
import csv
from sklearn import svm
import warnings
import random
from statistics import mode
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
random.seed(0)
warnings.filterwarnings('ignore')
def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-1]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)
def Cosine(a, b):#distance
    return distance.cosine(a,b)
def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return 1-isc
def most_common(lst):
    return(mode(lst))


def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["k",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=train_time+test_time
    for i in range(0,len(Arr)):
        print(i,len(Arr))
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)
def findDI(train_data,train_labels,metric,k):
    clss=set(train_labels)
    func=globals()[metric]
    radius={}
    th=0.5
    for cl in clss:
        indices=[i for i in range(0,len(train_labels)) if train_labels[i]==cl]
        elements=train_data[indices,:]
        distMat=pairwise_distances(elements,elements,metric=func)
        radius[cl]=np.nanmean(distMat)#/(distMat.shape[0]*distMat.shape[1])
    weights=[]
   
    classifier = NearestNeighbors(n_neighbors=k,metric=func)
    if len(train_labels)<k:
        classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        
    classifier.fit(train_data,train_labels)
    distances,neighVals=classifier.kneighbors(train_data,return_distance=True)
    print(neighVals.shape)
    for i in range(0,train_data.shape[0]):
        r=radius[train_labels[i]]
        ID=0
        neighs=np.array(neighVals[i,:],dtype=int)
        #print(neighs)
        neighLabels = [train_labels[i] for i in neighs]
           
        #neighLabels = [train_labels[i] for i in int(neighVals[i,:])]
        val=0
        c=1
        for nn,neigh in enumerate(neighLabels):
            if distances[i,nn]<r:
                c=c+1
                if neigh==train_labels[i]:
                    val=val+1
        ID=(val/c)
        ##############find weights using validity ###############
        #w=np.sum(val*(1/(distances[i]+alpha)))
        weights.append(ID)
    return weights
       

def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    averageMeanPrecison=[]
    kList=[5,15,30,45]
    func=globals()[metric]
    train_time=[]
    test_time=[]
    maxval=np.max(kList)
    var=np.zeros(train_data.shape[1])
    class_dist=[]
    label_dist=[]
    ##########Main piece of code############
    # split training data according to class distribution
    for k in kList:
        time1=time.time()
        print("k",k)
        weights=np.array(findDI(train_data,train_labels,metric,k))
        weights=weights/np.sum(weights)
        def weighted(distances):
            return 1/weights
            
        classifier = KNeighborsClassifier(n_neighbors=k,metric=func,weights=locals()['weighted'])

        if len(train_labels)<k:
            classifier = KNeighborsClassifier(n_neighbors=len(train_labels),metric=func,weights=locals()['weighted'])
        classifier.fit(train_data,train_labels)
        distances,neighs=classifier.kneighbors(test_data,return_distance=True)
        pred=[]
        time2=time.time()
        for t in range(0,test_data.shape[0]):
            weights2=[]
            dist=distances[t,:]
            neighs2=neighs[t,:]
            if dist[0]==dist[-1]:
                weights2=[1 for ii in range(0,k)]
            else:
                for j in range(0,len(dist)):
                    wj=((dist[-1]-dist[j])/(dist[-1]-dist[0])) * ((dist[-1]+dist[0])/(dist[-1]+dist[j]))
                    weights2.append(wj)
            
            neigh_data=train_data[neighs[t],:]
            neigh_dist=distances[t,:]
            neigh_labels=[train_labels[n] for n in neighs[t]]
            weightSum=0
            maxweight=-1
            predLabel=''
            for lbl in set(neigh_labels):
                weightSum=0
                #weightSum=np.sum([weights[ii]/(distances[j]+0.00001) for j,ii in enumerate(neighs) if train_labels[ii]==lbl])
               
                w1=np.nansum([1/(distances[t,j]+0.0001) for j,ii in enumerate(neighs[t]) if train_labels[ii]==lbl])
                w2=np.sum([1 for i in neighs[t] if train_labels[i]==lbl])/len(neighs[t])
                w3=np.nansum([weights2[j] for j,ii in enumerate(neighs[t]) if train_labels[ii]==lbl])
                
               
                weightSum=w1+w2+w3
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            pred.append(predLabel)

  
        y_pred.append(pred) 
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
      
    class_dict={}
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)

        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)

    #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Datasets April\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Datasets April\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = preprocessing.LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    classes=list(set(labels))
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "WPR-KNN\\Cancelled Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_WPRKNN.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=xTrain
            test_data=xTest
            train_labels=yTrain
            test_labels=yTest
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Nabeel\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nabeel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nabeel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nabeel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


b'Datasets April\\1_lung-cancer'
###############
k 5
(27, 5)
k 15
(27, 15)
k 30
(27, 27)
k 45
(27, 27)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+-----------------------+------------------------+----------------------+
|    k     | Accuracy |       Train Time      |       Test time        |      Total Time      |
+----------+----------+-----------------------+------------------------+----------------------+
|    5     |   0.75   |     0:00:00.005049    |     0:00:00.000970     |    0:00:00.005049    |
|    15    |   0.75   |     0:00:00.005017    |     0:00:00.000967     |    0:00:00.005017    |
|    30    |   0.75   |     0:00:00.005034    |        0:00:00         |    0:00:00.005034    |
|    45    |   0.75   |     0:00:00.006000    |        0:00:00         |    0:00:00.006000    |
| Average  |   0.75   |     0:00:00.005275    |     0:00:00.000484     |    0:00:00.002880    |
| Std Dev. |   0.0    | 0.0004188529080032567 | 0.0004842300123876

(28, 28)
k 45
(28, 28)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+-----------------------+-----------------------+
|    k     | Accuracy |      Train Time      |       Test time       |       Total Time      |
+----------+----------+----------------------+-----------------------+-----------------------+
|    5     |  0.3333  |    0:00:00.006000    |        0:00:00        |     0:00:00.006000    |
|    15    |  0.6667  |    0:00:00.005000    |     0:00:00.001000    |     0:00:00.005000    |
|    30    |  0.6667  |    0:00:00.006001    |     0:00:00.000999    |     0:00:00.006001    |
|    45    |  0.6667  |    0:00:00.006037    |        0:00:00        |     0:00:00.006037    |
| Average  |  0.5833  |    0:00:00.005759    |     0:00:00.000500    |     0:00:00.003130    |
| Std Dev. |  0.1443  | 0.000438668383958435 | 0.0004999042196923039 | 0.0026715061920710576 |
+----------+----------+----------------------+-------------

metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+-----------------------+----------------------+-----------------------+
|    k     | Accuracy |       Train Time      |      Test time       |       Total Time      |
+----------+----------+-----------------------+----------------------+-----------------------+
|    5     |   1.0    |     0:00:00.004999    |    0:00:00.000998    |     0:00:00.004999    |
|    15    |  0.6667  |     0:00:00.005000    |       0:00:00        |     0:00:00.005000    |
|    30    |  0.6667  |     0:00:00.006000    |       0:00:00        |     0:00:00.006000    |
|    45    |  0.6667  |     0:00:00.006001    |       0:00:00        |     0:00:00.006001    |
| Average  |   0.75   |     0:00:00.005500    |    0:00:00.000250    |     0:00:00.002875    |
| Std Dev. |  0.1443  | 0.0005005003796370776 | 0.000432361887818483 | 0.0026664706944635594 |
+----------+----------+-----------------------+----------------------+-----------------------+


###############
b'Datasets April\\arrhythmia'
###############
k 5
(405, 5)
k 15
(405, 15)
k 30
(405, 30)
k 45
(405, 45)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time     |
+----------+----------+----------------------+----------------------+---------------------+
|    5     |  0.5435  |    0:00:00.789959    |    0:00:00.005038    |    0:00:00.789959   |
|    15    |  0.5652  |    0:00:00.839961    |    0:00:00.011000    |    0:00:00.839961   |
|    30    |  0.5652  |    0:00:00.851002    |    0:00:00.016000    |    0:00:00.851002   |
|    45    |  0.5652  |    0:00:00.841034    |    0:00:00.018965    |    0:00:00.841034   |
| Average  |  0.5598  |    0:00:00.830489    |    0:00:00.012751    |    0:00:00.421620   |
| Std Dev

...

(499, 5)
k 15
(499, 15)
k 30
(499, 30)
k 45
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+---------------------+-----------------------+--------------------+
|    k     | Accuracy |      Train Time     |       Test time       |     Total Time     |
+----------+----------+---------------------+-----------------------+--------------------+
|    5     |   1.0    |    0:00:02.049000   |     0:00:00.005000    |   0:00:02.049000   |
|    15    |   1.0    |    0:00:02.543001   |     0:00:00.010000    |   0:00:02.543001   |
|    30    |   1.0    |    0:00:02.648000   |     0:00:00.011035    |   0:00:02.648000   |
|    45    |   1.0    |    0:00:02.942967   |     0:00:00.015999    |   0:00:02.942967   |
| Average  |   1.0    |    0:00:02.545742   |     0:00:00.010508    |   0:00:01.278125   |
| Std Dev. |   0.0    | 0.32210423219365675 | 0.0039056578135800994 | 1.2879189965162134 |
+----------+----------+---------------------+-------------------

(499, 5)
k 15
(499, 15)
k 30
(499, 30)
k 45
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+--------------------+----------------------+--------------------+
|    k     | Accuracy |     Train Time     |      Test time       |     Total Time     |
+----------+----------+--------------------+----------------------+--------------------+
|    5     |  0.9273  |   0:00:02.065038   |    0:00:00.005964    |   0:00:02.065038   |
|    15    |  0.9455  |   0:00:02.433001   |    0:00:00.008997    |   0:00:02.433001   |
|    30    |  0.9455  |   0:00:02.766002   |    0:00:00.016035    |   0:00:02.766002   |
|    45    |  0.9455  |   0:00:02.944963   |    0:00:00.015001    |   0:00:02.944963   |
| Average  |  0.9409  |   0:00:02.552251   |    0:00:00.011499    |   0:00:01.281875   |
| Std Dev. |  0.0079  | 0.3359722238518607 | 0.004175365543799726 | 1.2924018268461714 |
+----------+----------+--------------------+----------------------+---------------

(499, 5)
k 15
(499, 15)
k 30
(499, 30)
k 45
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+---------------------+-----------------------+-------------------+
|    k     | Accuracy |      Train Time     |       Test time       |     Total Time    |
+----------+----------+---------------------+-----------------------+-------------------+
|    5     |   1.0    |    0:00:02.070999   |     0:00:00.005004    |   0:00:02.070999  |
|    15    |  0.9818  |    0:00:02.438998   |     0:00:00.008034    |   0:00:02.438998  |
|    30    |  0.9818  |    0:00:02.802964   |     0:00:00.011000    |   0:00:02.802964  |
|    45    |  0.9455  |    0:00:02.908000   |     0:00:00.015000    |   0:00:02.908000  |
| Average  |  0.9773  |    0:00:02.555240   |     0:00:00.009759    |   0:00:01.282500  |
| Std Dev. |  0.0198  | 0.32932080279478854 | 0.0036944154175040634 | 1.293870590900356 |
+----------+----------+---------------------+-----------------------+----

(499, 5)
k 15
(499, 15)
k 30
(499, 30)
k 45
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+--------------------+-----------------------+--------------------+
|    k     | Accuracy |     Train Time     |       Test time       |     Total Time     |
+----------+----------+--------------------+-----------------------+--------------------+
|    5     |  0.9455  |   0:00:02.072000   |     0:00:00.006000    |   0:00:02.072000   |
|    15    |  0.9818  |   0:00:02.500003   |     0:00:00.008036    |   0:00:02.500003   |
|    30    |  0.9818  |   0:00:02.672962   |     0:00:00.012000    |   0:00:02.672962   |
|    45    |  0.9818  |   0:00:02.801035   |     0:00:00.015004    |   0:00:02.801035   |
| Average  |  0.9727  |   0:00:02.511500   |     0:00:00.010260    |   0:00:01.260880   |
| Std Dev. |  0.0157  | 0.2753149380608367 | 0.0034866466408817113 | 1.2656838622297524 |
+----------+----------+--------------------+-----------------------+-----

(499, 5)
k 15
(499, 15)
k 30
(499, 30)
k 45
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+---------------------+-----------------------+--------------------+
|    k     | Accuracy |      Train Time     |       Test time       |     Total Time     |
+----------+----------+---------------------+-----------------------+--------------------+
|    5     |   1.0    |    0:00:01.956001   |     0:00:00.004967    |   0:00:01.956001   |
|    15    |   1.0    |    0:00:02.584033   |     0:00:00.008964    |   0:00:02.584033   |
|    30    |  0.9455  |    0:00:02.662000   |     0:00:00.013001    |   0:00:02.662000   |
|    45    |  0.9273  |    0:00:02.757999   |     0:00:00.015000    |   0:00:02.757999   |
| Average  |  0.9682  |    0:00:02.490008   |     0:00:00.010483    |   0:00:01.250246   |
| Std Dev. |  0.0325  | 0.31440627891923395 | 0.0038562190438599883 | 1.2595414054247374 |
+----------+----------+---------------------+-------------------

In [2]:
test=[1,2,3,4]
print(test[0:-1])

[1, 2, 3]


In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
y_scores = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
roc_auc_score(y_true, y_scores)

0.5